In [25]:
import numpy as np
import pandas as pd

In [35]:
df = pd.read_csv('/content/sample_data/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [36]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [37]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
scaler = StandardScaler()

In [40]:
X = scaler.fit_transform(X)

In [41]:
X.shape

(768, 8)

In [42]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 2)

In [43]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [44]:
model = Sequential()
model.add(Dense(32,activation = 'relu',input_dim = 8))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
model.fit(X_train,y_train, batch_size=32, epochs = 100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.3661 - loss: 0.8309 - val_accuracy: 0.4221 - val_loss: 0.7415
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4998 - loss: 0.7199 - val_accuracy: 0.6364 - val_loss: 0.6607
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6615 - loss: 0.6352 - val_accuracy: 0.6948 - val_loss: 0.6069
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7071 - loss: 0.5944 - val_accuracy: 0.7273 - val_loss: 0.5718
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7163 - loss: 0.5655 - val_accuracy: 0.7273 - val_loss: 0.5488
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7408 - loss: 0.5368 - val_accuracy: 0.7208 - val_loss: 0.5308
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7510 - loss: 0.5160 - val_accuracy: 0.7273 - val_loss: 0.5181
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7582 - loss: 0.5106 - val_accuracy: 0.7403 - 

In [46]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [47]:
import kerastuner as kt

<ipython-input-47-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [58]:
def built_model(hp):
  model = Sequential()
  model.add(Dense(32, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta']), loss = 'binary_crossentropy', metrics= ['accuracy'])

  return model


In [59]:
tuner = kt.RandomSearch(built_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory = 'mydir')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [60]:
tuner.search(X_train,y_train,epochs=5,validation_data = (X_test,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 13s


In [62]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [63]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [67]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7459 - loss: 0.5353 - val_accuracy: 0.7662 - val_loss: 0.5036
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7515 - loss: 0.5126 - val_accuracy: 0.7792 - val_loss: 0.4892
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7799 - loss: 0.4758 - val_accuracy: 0.7727 - val_loss: 0.4815
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7611 - loss: 0.5043 - val_accuracy: 0.7662 - val_loss: 0.4745
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7903 - loss: 0.4602 - val_accuracy: 0.7792 - val_loss: 0.4708
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8169 - loss: 0.4417 - val_accuracy: 0.7792 - val_loss: 0.4678
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7853 - loss: 0.4654 - val_accuracy: 0.7792 - val_loss: 0.4630
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7816 - loss: 0.4691 - val_accuracy: 0.77

In [81]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72,activation = 'relu', input_dim = 8))

  for i in range(hp.Int('num_layers', min_value=1, max_value = 10)):
    model.add(Dense(72, activation = 'relu'))

  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = 'rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [82]:
tuner = kt.RandomSearch(build_model,
                      objective = 'val_accuracy',
                      max_trials=3,
                      directory = 'mydir',
                      project_name = 'num_layers')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [84]:
tuner.search(X_train,y_train, epochs=5, validation_data = (X_test,y_test))

Trial 3 Complete [00h 00m 27s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 29s


In [87]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [88]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [89]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7872 - loss: 0.4362 - val_accuracy: 0.7597 - val_loss: 0.5075
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8133 - loss: 0.4050 - val_accuracy: 0.7403 - val_loss: 0.4824
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7964 - loss: 0.4090 - val_accuracy: 0.7857 - val_loss: 0.4820
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8235 - loss: 0.3792 - val_accuracy: 0.7662 - val_loss: 0.5182
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7960 - loss: 0.4061 - val_accuracy: 0.7662 - val_loss: 0.4811
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8323 - loss: 0.3813 - val_accuracy: 0.7662 - val_loss: 0.4881
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8315 - loss: 0.3777 - val_accuracy: 0.7857 - val_loss: 0.4857
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8509 - loss: 0.3669 - val_accuracy: 0.7

In [109]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):

    if counter == 0:

      model.add(
          Dense(
                hp.Int('units' + str(i), min_value = 128, max_value = 128, step = 8),
                activation = hp.Choice('activation' + str(i), values = ['relu', 'tahh', 'sigmoid']),
                input_dim = 8
               )
          )
    else:
      model.add(
            Dense(
                  hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                  activation = hp.Choice('activation' + str(i), values = ['relu', 'tahh', 'sigmoid'])

                )
            )

      conter +=1

    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = hp.Choice('optimizer', values = ['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                      loss = 'binary_corssentropy',
                      metrics = ['accuracy'])

    return model



In [110]:
tune = kt.RandomSearch(build_model,
                       objective = 'val_accuracy',
                       max_trials = 3,
                       directory = 'mydir',
                       project_name = 'final'
                       )

In [111]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

In [112]:
tuner.get_best_hyperparameters ()[0].values

{'num_layers': 4}

In [113]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [115]:
model.fit(X_train,y_train, epochs = 200, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.7923 - loss: 0.4431 - val_accuracy: 0.7662 - val_loss: 0.4593
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8103 - loss: 0.3880 - val_accuracy: 0.7597 - val_loss: 0.4644
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8272 - loss: 0.3927 - val_accuracy: 0.7532 - val_loss: 0.4686
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8163 - loss: 0.3839 - val_accuracy: 0.7922 - val_loss: 0.4658
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8013 - loss: 0.3888 - val_accuracy: 0.7922 - val_loss: 0.4831
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8254 - loss: 0.3869 - val_accuracy: 0.7597 - val_loss: 0.4863
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8263 - loss: 0.3797 - val_accuracy: 0.7792 - val_loss: 0.4681
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8230 - loss: 0.3782 - val_accuracy: 0.77

In [ ]:
y.shape

(768,)